
add an env file with:
OPENAI_API_KEY = ...
LANGCHAIN_TRACING_V2=true
LANGCHAIN_API_KEY=...

to activate env cd to the project folder and run :

source env/bin/activate

In [7]:
pip install -r requirements.txt

  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import base64
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [9]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [10]:
chain = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)


In [13]:
image = encode_image("2.png")


msg = chain.invoke(
    [   AIMessage(
        content="You are a useful bot that is especially good at OCR from images for math formulas and graphs"
    ),
        HumanMessage(
            content=[
                {"type": "text", "text": "what is this graph about"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image}"
                    },
                },
            ]
        )
    ]
)
print(msg.content)

The graph depicts a curve in the xy-plane. The axes are labeled with "x" for the horizontal axis and "y" for the vertical axis. The curve appears to be a hyperbola, which is a type of conic section.

A hyperbola typically consists of two separate curves that open away from each other. The graph in the image shows only one branch of a hyperbola, which is opening to the right. Given the orientation and position, this branch corresponds to the positive half of a hyperbola that is centered at the origin.

The hyperbola is symmetric with respect to the x-axis, as indicated by the negative and positive y-values it takes on. The points where the curve intersects the axes are at (-1, 0) on the x-axis and (0, 1) and (0, -1) on the y-axis. These points suggest that the hyperbola may be represented by the equation of the form \( \frac{x^2}{a^2} - \frac{y^2}{b^2} = 1 \), where \( a \) and \( b \) are real numbers that determine the shape and position of the hyperbola.

The letter "b" with a green 

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser


chain = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1024)
prompt = PromptTemplate.from_template(
"""
You see the following foods in a product list:
{food}
Create a dish using only these ingredients and say what it is called. Only return the name of the dish.
No additional explanation or anything related. Just the name

Example:
'Pizza'
'Sushi'

Output:
"""
)
runnable = prompt | chain | StrOutputParser()

In [ ]:
dish = runnable.invoke({"food": msg.content})
print(dish)

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.images.generate(
  model="dall-e-3",
  prompt=f"A nice candlelight dinner with {dish} for two persons",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url


In [ ]:
print(image_url)